In [ ]:
# Set openai key
# Please gen and add `OPENAI_API_KEY` in the `.env` file in the current directory
# https://platform.openai.com/account/api-keys
# Please gen and add `HUGGINGFACEHUB_API_TOKEN` in the `.env` file in the current directory
# https://huggingface.co/settings/tokens
import os

from dotenv import load_dotenv

load_dotenv()

In [ ]:
LLM_MODEL = "gpt-3.5-turbo"

### Legacy Chains

#### LLM chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.9, model=LLM_MODEL)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Who is {person}?"
)

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
chain.run("Angus Young")

In [ ]:
chain.run("Freddie Mercury")

#### Sequential chain

In [ ]:
# prompt template 1: 
first_prompt = ChatPromptTemplate.from_template(
    "Who is {person}?"
)
# chain 1: input -> person and output -> person_info
first_chain = LLMChain(
    llm=llm,
    prompt=first_prompt, 
    output_key="person_info",
)

In [ ]:
# prompt template 2: 
second_prompt = ChatPromptTemplate.from_template(
    "Squeeze the description of the person into one sentence with maximum 20 words: {person_info}?"
)
# chain 2: input -> person_info and output -> short_info
second_chain = LLMChain(
    llm=llm,
    prompt=second_prompt, 
    output_key="short_info",
)

In [ ]:
# prompt template 3: 
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following sentence into the Spanish language: {short_info}?"
)
# chain 3: input -> short_info and output -> translated_info
third_chain = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="translated_info",
)

In [ ]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(
    chains=[first_chain, second_chain, third_chain],
    input_variables=["person"],
    output_variables=["person_info", "short_info", "translated_info"],
    verbose=True,
)

sequential_chain("Angus Young")

### New API Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model=LLM_MODEL)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "When {person} was born?"
)

chain = prompt | llm

In [ ]:
response = chain.invoke({"person": "Angus Young"})
response.content

In [ ]:
response = chain.invoke({"person": "Freddie Mercury"})
response.content

#### Output parsers

In [ ]:
from langchain.output_parsers import DatetimeOutputParser

datetim_output_parser = DatetimeOutputParser()

prompt = ChatPromptTemplate.from_template(
    """
    Give answer: When {person} was born?
    Format instruction: {format_instruction}
    """
)

In [ ]:
chain = prompt | llm | datetim_output_parser

In [ ]:
chain.invoke({"person": "Angus Young", "format_instruction": datetim_output_parser.get_format_instructions()})

#### Use 2 output parsers

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """
    {question}
    {format_instruction}
    """
)

chain = prompt | llm | datetim_output_parser

In [ ]:
# Answer has wrong format
chain.invoke({"question": "When was openAI launched?", "format_instruction": datetim_output_parser.get_format_instructions()})

In [ ]:
from langchain.output_parsers import OutputFixingParser

fix_parser = OutputFixingParser.from_llm(parser=datetim_output_parser, llm=ChatOpenAI(model=LLM_MODEL))

chain = prompt | llm | (datetim_output_parser and fix_parser)

In [ ]:
chain.invoke({"question": "When was openAI launched?", "format_instruction": datetim_output_parser.get_format_instructions()})

#### Multiple chains

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt1 = ChatPromptTemplate.from_template("Write a huge story about {story_object}")
ll1 = ChatOpenAI(model=LLM_MODEL)

chain1 = prompt1 | ll1

In [ ]:
prompt2 = ChatPromptTemplate.from_template(
    """
    You are pirate. Make a short review of provided story.
    Story: ```{story}```
    """
)
ll2 = ChatOpenAI(model=LLM_MODEL)
chain2 = {"story": chain1} | prompt2 | ll2

In [ ]:
print(chain2.invoke({"story_object": "pizza"}).content)

In [ ]:
prompt3 = ChatPromptTemplate.from_template("```{text}```")
sumorization_model = HuggingFaceHub(repo_id="facebook/bart-large-cnn")
chain3 = {"text": chain2} | prompt3 | sumorization_model

In [ ]:
print(chain3.invoke({"story_object": "pizza"}))